In [104]:
import sys, importlib
from Q_values import Q_values
from NeuralNet import NeuralNet
importlib.reload(sys.modules['Q_values'])
importlib.reload(sys.modules['NeuralNet'])
import numpy as np
from generate_game import generate_game
print("-------------------")
nn = NeuralNet([3,4,2])
for idx in range(len(nn.W_bias)):
    print(idx)
    print("W" + str(np.asarray(nn.W_bias[idx][0]).shape))
    print("b" + str(np.asarray(nn.W_bias[idx][1]).shape))
    print()
print("-------------------")
W1 = np.asarray([
    [1,1,1,1],
    [1,2,1,2],
    [1,3,3,1]
])
b1 = np.ones(4)

W2 = np.asarray(
    [[1,2],[2,1],[2,3],[1,3]])
b2 = np.zeros((2))
nn.W_bias=[(W1,b1), (W2,b2)]
(output, nv) = nn.feedforward([1,2,3])

for n in nv:
    print(n)
print("-------------------")

delta=np.asarray([1,0])
r = nn.partial_derivative( [(W1, b1), (W2, b2)], delta, nv )
id = 1
for i in r:
    print("d_w"+str(id))
    print(i[0])
    print("d_b"+str(id))
    print(i[1])
    print("d_a"+str(id))
    print(i[2])
    id+=1

-------------------
0
W(3, 4)
b(4,)

1
W(4, 2)
b(2,)

-------------------
(array([1, 2, 3]), array([1, 2, 3]))
(array([ 7., 15., 13.,  9.]), array([ 7., 15., 13.,  9.]))
(array([72., 95.]), array([72., 95.]))
[72. 95.]
-------------------
d_w1
[[1. 2. 2. 1.]
 [2. 4. 4. 2.]
 [3. 6. 6. 3.]]
d_b1
[1. 2. 2. 1.]
d_a1
[[1. 2. 2. 1.]]
d_w2
[[ 7.  0.]
 [15.  0.]
 [13.  0.]
 [ 9.  0.]]
d_b2
[1. 0.]
d_a2
[[1 0]]


In [105]:
eta = 1
a_agent = 2
R = 4
W1 = np.asarray([
    [1.,1.,1.],
    [1.,2.,1.]
])
b1 = np.ones(3)
W2 = np.asarray([
    [1.,1.,1.],
    [1.,2.,1.],
    [2.,2.,3.]
])
b2 = np.ones(3)
x = [1,2]
x1 = np.asarray(x).dot(W1)+b1
Q = x1.dot(W2)+b2
delta = R - Q[a_agent]

delta_W2 = eta * delta * x1
delta_W1 = eta * np.outer(x, delta * W2[:, a_agent] * (x1 > 0))

nn = NeuralNet([2,3,3,2])
nn.W_bias=[(W1,b1),(W2,b2)]
(nno, nnv) = nn.feedforward(x)
print("nn output")
print(nno)
print("Q")
print(Q)
print("-------------------------")

delta_v = np.zeros(len(b2))
delta_v[a_agent] = delta
dw = nn.partial_derivative(nn.W_bias,-delta_v,nnv)
print("nn deriv")
print(f"w1 {dw[0][0]} \n b1 {dw[0][1]}")
print(f"w2 {dw[1][0]} \n b2 {dw[1][1]}")
print("calculate deriv")
print(f"d W1 {delta_W1} \n b1 {delta*W2[:,a_agent]*(x1>0)}\n "
      f"d W2 {delta_W2} \n b2 {delta}")
print("-------------------")

# nn.update(eta,nnv, -delta_v)
nn.momentum_gradient_decent(eta, -delta_v, nnv)
# nn.momentum_gradient_decent(eta, -delta_v, nnv)
print("update")
idx = 0
for wb in nn.W_bias:
    print(f"{idx+1} w\n {wb[0]} \n b\n {wb[1]}  ")
    idx+=1

print("calculate update")
b1 = b1 + eta * delta * W2[:, a_agent] * (x1>0)
W1 = W1 + eta * delta_W1

W2[:, a_agent] = W2[:, a_agent] + eta * delta_W2
b2[a_agent] = b2[a_agent] + eta * delta
print(f"{1} w\n {W1} \n b \n{b1}  ")
print(f"{2} w\n {W2} \n b\n {b2}  ")


nn output
[19. 25. 23.]
Q
[19. 25. 23.]
-------------------------
nn deriv
w1 [[ 19.  19.  57.]
 [ 38.  38. 114.]] 
 b1 [19. 19. 57.]
w2 [[  0.   0.  76.]
 [  0.   0. 114.]
 [  0.   0.  76.]] 
 b2 [ 0.  0. 19.]
calculate deriv
d W1 [[ -19.  -19.  -57.]
 [ -38.  -38. -114.]] 
 b1 [-19. -19. -57.]
 d W2 [ -76. -114.  -76.] 
 b2 -19.0
-------------------
update
1 w
 [[ -18.1  -18.1  -56.1]
 [ -37.1  -36.1 -113.1]] 
 b
 [-18.1 -18.1 -56.1]  
2 w
 [[   0.9    0.9  -75.1]
 [   0.9    1.9 -113.1]
 [   1.9    1.9  -73.1]] 
 b
 [  0.9   0.9 -18.1]  
calculate update
1 w
 [[ -18.  -18.  -56.]
 [ -37.  -36. -113.]] 
 b 
[-18. -18. -56.]  
2 w
 [[   1.    1.  -75.]
 [   1.    2. -113.]
 [   2.    2.  -73.]] 
 b
 [  1.   1. -18.]  
